In [ ]:
import pandas as pd
import numpy as np

In [ ]:
item_col = ['id' , 'created_at', 'updated_at', 'deleted_at', 'name', 'price', 'site_name', 'weight', 'category', 'date', 'cnt', 'unit', 'bundle', 'image_src', 'size']
kur_col = ['category_name', 'prd_name', 'price', 'weight']

In [ ]:
import psycopg2

try: 
    conn = psycopg2.connect(host = "kurly-postgresql.c9u9ipgkij2c.ap-northeast-2.rds.amazonaws.com"
                            , dbname = "postgres"
                            , user = "postgres"
                            , password = "kurly2022"
                            , port = 5432)
except:
    print("연결안됨")
    
    
cur = conn.cursor()
cur.execute("select * from public.kurly_list;")
rows = cur.fetchall()

df = pd.DataFrame(rows, columns = kur_col)

In [ ]:
kur_col

['category_name', 'prd_name', 'price', 'weight']

In [ ]:
df.category_name.unique()

array(['사과', '김치'], dtype=object)

In [ ]:
df1 = df[df['category_name'] == '사과'].reset_index(drop=True)
df2 = df[df['category_name'] == '김치'].reset_index(drop=True)

In [ ]:
def cal_g(df, pcol, wcol):
    df[pcol] = df[pcol].astype('int')
    df[wcol] = df[wcol].astype('int')
    df['perp'] = ((df[pcol]*100)/df[wcol]).round(2)
    return df

In [ ]:
df1 = cal_g(df1, 'price', 'weight')
df2 = cal_g(df2, 'price', 'weight')

In [ ]:
df1.groupby(['category_name'])['perp'].describe()[['min', 'mean', 'max']]

,min,mean,max
category_name,,,
사과,498.0,1052.764,2560.0


In [ ]:
df2.groupby(['category_name'])['perp'].describe()[['min', 'mean', 'max']]

,min,mean,max
category_name,,,
김치,4.31,3252.023378,33000.0


In [ ]:
# df['price2'] = df['price'].str.replace(',', '').astype('int64') 
# df['w1'] = df['weight'].str.replace('kg', '').str.replace('g','').str.replace('g','').str.replace('.','')
# df['w2'] = np.where(df['w1'] == '', 0 , df['w1'])
# df['w2'] = df['w2'].astype('int64')
# df['wc'] = np.where(df['unit'] == 'kg', df['w2']*100, df['w2'])
# df['perp'] = ((df['price2']*100)/df['wc']).round(2)
# #n_col = item_col + ['perp']
# #df = df[n_col]

In [ ]:
from collections import Counter

def detect_outliers(df, n, features):
    outlier_indices = []
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        
        outlier_step = 1.5 * IQR
        
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k, v in outlier_indices.items() if v > n)
    
    return multiple_outliers
        
Outliers_to_drop = detect_outliers(df1, 2, ['perp'])
df1 = df1.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [ ]:
Outliers_to_drop = detect_outliers(df2, 2, ['perp'])
df2 = df2.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [ ]:
df1['perp'].describe()[['min', 'mean', 'max']].reset_index().T

,0,1,2
index,min,mean,max
perp,498,1052.76,2560


In [ ]:
df2['perp'].describe()[['min', 'mean', 'max']].reset_index().T

,0,1,2
index,min,mean,max
perp,4.31,3252.02,33000


In [ ]:
cur.execute("insert into price_sumry values "20220823", "kurly", "사과", 498, 1052
            76, 2560";")

SyntaxError: invalid syntax (<ipython-input-87-68f643a462c5>, line 1)

Ver2

In [1]:
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime

In [2]:
item_col = ['id' , 'created_at', 'updated_at', 'deleted_at', 'name', 'price', 'site_name', 'weight', 'category_name', 'date', 'cnt', 'unit', 'bundle', 'image_src', 'size']
kur_col = ['category_name', 'prd_name', 'price', 'weight']

In [5]:
# select 선택
def select_df(sql, col):
  try:
      conn = psycopg2.connect(host = "kurly-postgresql.c9u9ipgkij2c.ap-northeast-2.rds.amazonaws.com"
                              , dbname = "postgres"
                              , user = "postgres"
                              , password = "kurly2022"
                              , port = 5432)
  except:
      print("연결안됨")  
    
  cur = conn.cursor()
  cur.execute(sql)
  rows = cur.fetchall()

  df = pd.DataFrame(rows, columns = col)
  return df


In [8]:
# commit 입력
def commit_df(sql):
  try:
      conn = psycopg2.connect(host = "kurly-postgresql.c9u9ipgkij2c.ap-northeast-2.rds.amazonaws.com"
                              , dbname = "postgres"
                              , user = "postgres"
                              , password = "kurly2022"
                              , port = 5432)
  except:
      print("연결안됨")  
    
  cur = conn.cursor()
  sql = cur.execute(sql)
  rows = cur.fetchall()

  print("record inserted")

In [9]:
# 이상치 제거
def detect_outliers(df, n, features):
    outlier_indices = []
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
       
        outlier_step = 1.5 * IQR
       
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k, v in outlier_indices.items() if v > n)
   
    return multiple_outliers
       
def remove_outliers(df, cnt, col):
	Outliers_to_drop = detect_outliers(df, cnt, list(col))
	df = df1.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)
	return df

In [52]:
# 가격연산
def cal_g(df, pcol, wcol):
    if 'unit' in df.columns:
      df[wcol] = np.where(df['unit'] == 'kg', df[wcol]*100, df[wcol])
      df[pcol] = df[pcol].astype('int')
      df[wcol] = df[wcol].astype('int')
      df['perp1'] = ((df[pcol]*100)/df[wcol]).round(2)
      df['perp'] = np.where(df['perp1']==np.inf, 0, df['perp1'])
    else:
      df[pcol] = df[pcol].astype('int')
      df[wcol] = df[wcol].astype('int')
      df['perp1'] = ((df[pcol]*100)/df[wcol]).round(2)
      df['perp'] = np.where(df['perp1']==np.inf, 0, df['perp1'])       
    return df

In [54]:
# 컬리 & 경쟁사 데이터
li_col = [['price_info', item_col, 'weight', 'price' ], ['kurly_list', kur_col, 'weight', 'price']]

for d, c, w, p in li_col:
  sql = 'select * from public.' + d
  df = select_df(sql, c)
  cat = list(df.category_name.unique())
  for n in cat:
    df1 = df[df['category_name'] == n].reset_index(drop=True)
    print(d,c,w,p)
    df1 = cal_g(df1, p, w)
    res = remove_outliers(df1, 2, 'perp')
    mmm = list(res['perp'].describe()[['min','mean','max']].reset_index().T.reset_index())
    date = str(datetime.today().year)+str(datetime.today().month).zfill(2)+str(datetime.today().day).zfill(2)
    isrt = list(date) + mmm
    sql = "insert into price_sumry values (isrt[0], d, isrt[1], isrt[2], isrt[3], isrt[4]);"
    commit_df(sql)

price_info ['id', 'created_at', 'updated_at', 'deleted_at', 'name', 'price', 'site_name', 'weight', 'category_name', 'date', 'cnt', 'unit', 'bundle', 'image_src', 'size'] weight price


KeyError: ignored